In [ ]:
import psycopg2
import pandas as pd
import time
import seaborn as sns 
sns.set();
import matplotlib.pyplot as plt
import numpy as np
import mysql.connector
from influxdb import InfluxDBClient

### TimescaleDB Connection

In [65]:
# Luodaan yhteys
timescaledb = psycopg2.connect(user = "postgres",
                              password = "password",
                              host = "172.17.0.1",
                              port = "5432",
                              dbname = "iiwari")

# Koko höskän luonti
# real    10m26,917s
# user    0m2,250s
# sys     0m5,637s

### MariaDB Connection

In [66]:
# En muistanut kellottaa, mutta populoinnissa meni noin.30min

mariadb = mysql.connector.connect(
    host="172.17.0.4",
    user="root",
    passwd="insert-password-here",
    database="iiwari_org"
)

#### InfluxDB Connection

In [77]:

# koko höskän luonti
# real    119m57,521s
# user    0m3,748s
# sys     0m7,392s
influxdb = InfluxDBClient(host='172.17.0.5', 
                        port=8086, 
                        username='admin', 
                        password='teamfox',
                        database="iiwari_org")

# query = "SELECT * FROM SensorData limit 140000"
# for i in range(20):
#     result = influxdb.query(query)
#     cpu_points = list(result.get_points())
# df = pd.DataFrame(list(cpu_points))

In [78]:
connections = [timescaledb, mariadb]

def speed_test(connection):
    times = []
    for round in range(5):
        cursor = connection.cursor()
        start = time.time()
        cursor.execute("SELECT * FROM SensorData limit 1400")
        cursor.fetchall()
        times.append(time.time()-start)
#         print(time.time() - start)

    return times

def scalability_test(connection):
    times = []
    for rows in range(0, 1400, 100):
        cursor = connection.cursor()
        start = time.time()
        statement = "SELECT * FROM SensorData limit " + str(rows)+";"
        cursor.execute(statement)
        cursor.fetchall()
        times.append(time.time()-start)
#         print(time.time()-start)

    return times

def speed_test_and_to_dataframe(connection):
    times = []
    for round in range(5):
        cursor = connection.cursor()
        start = time.time()
        cursor.execute("SELECT * FROM SensorData limit 1400")
        asd = pd.DataFrame(cursor.fetchall())
        times.append(time.time()-start)
#         print(time.time() - start)
    return times

def speed_test_condition(connection):
    times = []
    for round in range(2):
        cursor = connection.cursor()
        start = time.time()
        cursor.execute("SELECT * FROM SensorData WHERE node_id = 3200")
        asd = pd.DataFrame(cursor.fetchall())
        times.append(time.time()-start)
#         print(time.time() - start)
    return times

db_speed_times = []
db_scalability_times = []
db_to_dataframe_tiems = []
db_speed_condition_times = []
for connection in connections:
    db_speed_times.append(speed_test(connection))
    db_scalability_times.append(scalability_test(connection))
    db_to_dataframe_tiems.append(speed_test_and_to_dataframe(connection))
    db_speed_condition_times.append(speed_test_condition(connection))

In [ ]:

def plot_benchmarks(db_scalability_times, db_speed_times, db_to_dataframe_tiems, db_speed_condition_times):
    Databases = ["TimescaleSB", "MariaDB"]
    fig, axes = plt.subplots(2,2, figsize=(16, 11))
    for i in range(len(db_scalability_times)):
        
        sns.lineplot(ax=axes[0][0],x=[x for x in range(len(db_speed_times[i]))], y=db_speed_times[i]).set_title("SELECT * FROM SensorData limit 1400000")
        axes[0][0].set_ylabel("Time s")
        axes[0][0].xaxis.set_major_locator(plt.MultipleLocator(1))
        axes[0][0].set_xlabel("Number of iterations")
        
        sns.lineplot(ax=axes[0][1],x=[x for x in range(len(db_to_dataframe_tiems[i]))], y=db_to_dataframe_tiems[i]).set_title("SELECT * FROM SensorData limit 1400000 + dataframe")
        axes[0][1].set_ylabel("Time s")
        axes[0][1].xaxis.set_major_locator(plt.MultipleLocator(1))
        axes[0][1].set_xlabel("Number of iterations")
        
        sns.lineplot(ax=axes[1][0],x=[x for x in range(len(db_speed_condition_times[i]))], y=db_speed_condition_times[i]).set_title("SELECT * FROM SensorData WHERE node_id = 3200 + dataframe")
        axes[1][0].set_ylabel("Time s")
        axes[1][0].xaxis.set_major_locator(plt.MultipleLocator(1))
        axes[1][0].set_xlabel("Number of iterations")
        

        sns.lineplot(ax=axes[1][1],x=[x for x in range(len(db_scalability_times[i]))], y=db_scalability_times[i]).set_title("SELECT * FROM SensorData LIMIT")
        axes[1][1].xaxis.set_major_locator(plt.MultipleLocator(1))
        axes[1][1].set_ylabel("Time s")
        axes[1][1].set_xlabel("Number of rows *100,000")
    
    plt.suptitle('Benchmarks ',fontsize=20)
    axes[0][0].legend(title='Database', loc='upper center', labels=Databases)
    axes[0][1].legend(title='Database', loc='upper center', labels=Databases)
    axes[1][0].legend(title='Database', loc='upper center', labels=Databases)
    axes[1][1].legend(title='Database', loc='upper center', labels=Databases)

    plt.show()
    
plot_benchmarks(db_scalability_times, db_speed_times, db_to_dataframe_tiems, db_speed_condition_times)